<a href="https://colab.research.google.com/github/DilshodbekMX/GoogleColab/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch